## 학습한 모델에 대한 추론 테스트: 학습이 정상적으로 진행되었는지 확인

In [1]:
import os
import json

def read_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)
        

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"
test_option = "postop" # binary or delta or postop

In [3]:
from llamafactory.chat import ChatModel

args = dict(
  model_name_or_path="Qwen/Qwen2.5-VL-3B-Instruct", # or full path to snapshot
  adapter_name_or_path=f"saves/qwen2.5_3b_{test_option}", # load the saved LoRA adapter
  template="qwen2_vl",  # same to the one in training
  finetuning_type="lora", # same to the one in training
  cache_dir="/home/nas5/.cache/huggingface/hub "
)
chat_model = ChatModel(args)

/home/seungbinyang/anaconda3/envs/dcm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[INFO|tokenization_utils_base.py:2095] 2025-11-28 08:52:44,573 >> loading file vocab.json from cache at /home/nas5/.cache/huggingface/hub /models--Qwen--Qwen2.5-VL-3B-Instruct/snapshots/66285546d2b821cf421d4f5eb2576359d3770cd3/vocab.json
[INFO|tokenization_utils_base.py:2095] 2025-11-28 08:52:44,576 >> loading file merges.txt from cache at /home/nas5/.cache/huggingface/hub /models--Qwen--Qwen2.5-VL-3B-Instruct/snapshots/66285546d2b821cf421d4f5eb2576359d3770cd3/merges.txt
[INFO|tokenization_utils_base.py:2095] 2025-11-28 08:52:44,577 >> loading file tokenizer.json from cache at /home/nas5/.cache/huggingface/hub /models--Qwen--Qwen2.5-VL-3B-Instruct/snapshots/66285546d2b821cf421d4f5eb2576359d3770cd3/tokenizer.json
[INFO|

[INFO|2025-11-28 08:52:49] llamafactory.model.model_utils.kv_cache:143 >> KV cache is enabled for faster generation.


[WARNING|logging.py:328] 2025-11-28 08:52:49,405 >> `torch_dtype` is deprecated! Use `dtype` instead!
[INFO|modeling_utils.py:1172] 2025-11-28 08:52:49,409 >> loading weights file model.safetensors from cache at /home/nas5/.cache/huggingface/hub /models--Qwen--Qwen2.5-VL-3B-Instruct/snapshots/66285546d2b821cf421d4f5eb2576359d3770cd3/model.safetensors.index.json
[INFO|modeling_utils.py:2341] 2025-11-28 08:52:49,414 >> Instantiating Qwen2_5_VLForConditionalGeneration model under default dtype torch.bfloat16.
[INFO|configuration_utils.py:986] 2025-11-28 08:52:49,417 >> Generate config GenerationConfig {
  "bos_token_id": 151643,
  "eos_token_id": 151645
}

[INFO|modeling_utils.py:2341] 2025-11-28 08:52:49,418 >> Instantiating Qwen2_5_VisionTransformerPretrainedModel model under default dtype torch.bfloat16.
[INFO|modeling_utils.py:2341] 2025-11-28 08:52:49,429 >> Instantiating Qwen2_5_VLTextModel model under default dtype torch.bfloat16.
Loading checkpoint shards: 100%|██████████| 2/2 [00

[INFO|2025-11-28 08:52:52] llamafactory.model.model_utils.attention:143 >> Using torch SDPA for faster training and inference.
[INFO|2025-11-28 08:52:52] llamafactory.model.adapter:143 >> Merged 1 adapter(s).
[INFO|2025-11-28 08:52:52] llamafactory.model.adapter:143 >> Loaded adapter(s): saves/qwen2.5_3b_postop
[INFO|2025-11-28 08:52:52] llamafactory.model.loader:143 >> all params: 3,754,622,976


In [4]:
test_dataset = read_json_file(f"data/dcm_{test_option}_only.json")
system_prompt = test_dataset[0]['messages'][0]['content'] # LLaMA Factory 특성상, 시스템 프롬프트 분리
input_messages = [test_dataset[0]['messages'][1]]
output = test_dataset[0]['messages'][2]['content']
images = [p.replace("../../","../") for p in test_dataset[0]['images']] # 테스트 환경 경로에 맞게 전처리

print("Test Sample:")
system_prompt, input_messages, output

Test Sample:


('\n<PROMPT>\n<ROLE>\nYou are an expert clinical reasoning AI specializing in neurosurgery, radiology, and neurology. You have deep knowledge of Degenerative Cervical Myelopathy (DCM).\n</ROLE>\n\n<CLINICAL_SCENARIO>\nYou will be provided with data for a patient with Degenerative Cervical Myelopathy (DCM) who has undergone surgical decompression. The goal is to predict their functional outcome one year after the surgery.\nAll the data you receive was collected before the surgery, with the exception of the operative note, which is generated during the surgery. The full spectrum of potentially available data includes:\n- Imaging Data: MR T2 Sagittal, MR T2 Axial.\n- Text Data: Radiology reports for the MR/XR, the pre-operative admission note, and the operative note.\n- Tabular Data: Patient demographics, past medical history, pre-operative lab results, and pre-operative symptoms.\nFor any given patient, you will receive a subset of this data, as specified in the task. Your analysis must 

### 성능 테스트 목적이 아니므로 샘플 한 개로 학습때 먹인 label 형태가 나오는지 테스트

In [5]:
response = chat_model.chat(system=system_prompt, messages=input_messages, images=images)[0].response_text

In [6]:
clean_response = response.replace("```json", "").replace("```", "").strip()
response_json = json.loads(clean_response)
print(response_json)

{'rationale': 'Based on the provided imaging and clinical data, the predicted 1-year post-operative JOA score is 15.', 'score': 15}


In [7]:
output

'{"rationale": "Based on the provided imaging and clinical data, the predicted post-operative JOA score is 14.0.", "score": 14}'